## Change these two to suit your experiment

In [1]:
csv_dir = "D:/Coryn/Dispase/12_8_22_Dispase_24h_MIG_TMAO_n=3/object-csvs"
output_file = "D:/Coryn/Dispase/12_8_22_Dispase_24h_MIG_TMAO_n=3/FinalCollated_test-delete-from-WG.csv"


In [2]:
import pandas as pd
import sys
import os
import argparse
input_CSVs = os.listdir(csv_dir)

first_file = os.path.join(csv_dir, input_CSVs[0])

first_filename = input_CSVs[0]


In [3]:
df = pd.read_csv(first_file)
df.head()

,object_id,timestep,labelimage_oid,User Label,Predicted Class,Probability of Background,Probability of Fragment,Probability of Ring,Bounding Box Minimum_0,Bounding Box Minimum_1,Bounding Box Maximum_0,Bounding Box Maximum_1,Center of the object_0,Center of the object_1,Size in pixels,Object Area
0,0,0,1,0,Fragment,0.44,0.45,0.11,0.0,0.0,304.0,304.0,148.12401,149.561680,86920.0,86920.0
1,1,0,2,0,Fragment,0.00,0.62,0.38,139.0,24.0,142.0,26.0,139.80000,24.400000,5.0,5.0
2,2,0,3,0,Fragment,0.00,0.61,0.39,129.0,25.0,134.0,33.0,130.86363,27.954546,22.0,22.0
3,3,0,4,0,Fragment,0.02,0.66,0.32,140.0,28.0,144.0,30.0,141.50000,28.333334,6.0,6.0
4,4,0,5,0,Fragment,0.00,0.55,0.45,103.0,31.0,106.0,33.0,104.00000,31.500000,6.0,6.0


## Add image info to objects

In [4]:
df["Image"] = first_filename[:-4]
df.rename({"Predicted Class" : "Predicted_Class"},
              axis = "columns",
              inplace = True
             )
df["Antibody"]             = df.Image.str.split("_", expand = True)[0]
df["Drug"]                 = df.Image.str.split("_", expand = True)[1]
df["Technical_Replicate"]  = df.Image.str.split("_", expand = True)[2]
df["Biological_Replicate"] = df.Image.str.split("_", expand = True)[3]
#df["Count"] = 1
df.head()

,object_id,timestep,labelimage_oid,User Label,Predicted_Class,Probability of Background,Probability of Fragment,Probability of Ring,Bounding Box Minimum_0,Bounding Box Minimum_1,...,Center of the object_0,Center of the object_1,Size in pixels,Object Area,Image,Antibody,Drug,Technical_Replicate,Biological_Replicate,Count
0,0,0,1,0,Fragment,0.44,0.45,0.11,0.0,0.0,...,148.12401,149.561680,86920.0,86920.0,NH_MIG_rep1_001_table,NH,MIG,rep1,001,1
1,1,0,2,0,Fragment,0.00,0.62,0.38,139.0,24.0,...,139.80000,24.400000,5.0,5.0,NH_MIG_rep1_001_table,NH,MIG,rep1,001,1
2,2,0,3,0,Fragment,0.00,0.61,0.39,129.0,25.0,...,130.86363,27.954546,22.0,22.0,NH_MIG_rep1_001_table,NH,MIG,rep1,001,1
3,3,0,4,0,Fragment,0.02,0.66,0.32,140.0,28.0,...,141.50000,28.333334,6.0,6.0,NH_MIG_rep1_001_table,NH,MIG,rep1,001,1
4,4,0,5,0,Fragment,0.00,0.55,0.45,103.0,31.0,...,104.00000,31.500000,6.0,6.0,NH_MIG_rep1_001_table,NH,MIG,rep1,001,1


In [5]:
def ProcessDF(file_path, file_name, df):
    # Remove the last four characters which should be `.csv`
    df["Image"] = file_name[:-4]
    
    # Rename to remove space
    df.rename({"Predicted Class" : "Predicted_Class"},
              axis = "columns",
              inplace = True
             )
    df["Antibody"]             = df.Image.str.split("_", expand = True)[0]
    df["Drug"]                 = df.Image.str.split("_", expand = True)[1]
    df["Technical_Replicate"]  = df.Image.str.split("_", expand = True)[2]
    df["Biological_Replicate"] = df.Image.str.split("_", expand = True)[3]
    
    #df["Count"] = 1
    return df

In [6]:
df_concat = df
for f in input_CSVs[1:]:
    
    f_path = os.path.join(csv_dir, f)
    
    df_temp = pd.read_csv(f_path)
    
    df_with_name = ProcessDF(f_path, f, df_temp)
    
    df_concat = pd.concat([df_concat, df_with_name])

In [7]:
df_filtered = df_concat.loc[df_concat["Predicted_Class"] == "Fragment"]

In [11]:
df_filtered.groupby(["Antibody", "Drug", "Biological_Replicate", "Technical_Replicate"]).size().reset_index(name="Counts")

,Antibody,Drug,Biological_Replicate,Technical_Replicate,counts
0,NH,MIG,001,rep1,169
1,NH,MIG,001,rep2,212
2,NH,MIG,001,rep3,175
3,NH,MIG,002,rep1,178
4,NH,MIG,002,rep2,238
5,NH,MIG,002,rep3,173
6,NH,MIG,003,rep1,146
7,NH,MIG,003,rep2,214
8,NH,MIG,003,rep3,170
9,NH,TMAO,001,rep1,78


In [14]:
(df_filtered.groupby(["Antibody", "Drug", "Biological_Replicate", "Technical_Replicate"])
 .size()
 .reset_index(name="Counts")
 .to_csv(output_file, index=False)
)